In [25]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.utils import to_categorical
import csv

def load_data(x_path, mode = 'train'):
    with open(x_path, 'r') as f:
        csv_f = csv.reader(f)
        X, Y = [], []
        for row in csv_f:
            row[1] = row[1].split()
            Y.append(row[0])
            X.append(row[1])
        del X[0], Y[0]
        X = np.array(X)
        Y = np.array(Y)
        
        X = X.astype(np.float)
        Y = Y.astype(np.int)
        X = X.reshape((X.shape[0], 48, 48, 1))
        X = X / 255 # normalize
        Y = Y.reshape((Y.shape[0]))
        if mode == 'train':
            Y = to_categorical(Y, 7)
    return X, Y

In [26]:
x_train, y_train = load_data('data/train.csv', mode = 'train')
x_test, _ = load_data('data/test.csv', mode = 'test')

In [66]:
def build_model(dropout_rate):
    model = Sequential()
    # Create CN layer 1  
    model.add(Conv2D(filters=16,
                     kernel_size=(6,6),
                     padding='same',
                     input_shape=(48,48,1),
                     activation='relu'))
    # Create Max-Pool 1  
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Create CN layer 2  
    model.add(Conv2D(filters=32,
                     kernel_size=(3,3),
                     padding='same',
                     activation='relu'))

    # Create Max-Pool 2  
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Add Dropout layer  
    model.add(Dropout(dropout_rate))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(7, activation='softmax'))
    model.summary()
    print("")
    return model

In [67]:
def train(model, x_train, y_train, batch_size, epochs, validation_split):
    # 定義訓練方式  
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  

    # 開始訓練  
    train_history = model.fit(x=x_train,  
                              y=y_train, validation_split=validation_split,  
                              epochs=epochs, batch_size=batch_size, verbose=1)

In [68]:
model = build_model(dropout_rate = 0.5)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 48, 48, 16)        592       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 4608)              0         
_________________________________________________________________
dense_27 (Dense)             (None, 128)               589952    
__________

In [70]:
train(model, x_train, y_train, 512, 20, 0.3)

Train on 20096 samples, validate on 8613 samples
Epoch 1/20
20096/20096 [==============================] - 53s 3ms/step - loss: 1.8603 - acc: 0.2214 - val_loss: 1.7877 - val_acc: 0.2543
Epoch 2/20
20096/20096 [==============================] - 52s 3ms/step - loss: 1.7913 - acc: 0.2633 - val_loss: 1.6958 - val_acc: 0.3458
Epoch 3/20
20096/20096 [==============================] - 56s 3ms/step - loss: 1.7163 - acc: 0.3094 - val_loss: 1.6141 - val_acc: 0.3822
Epoch 4/20
20096/20096 [==============================] - 58s 3ms/step - loss: 1.6637 - acc: 0.3414 - val_loss: 1.5682 - val_acc: 0.4100
Epoch 5/20
20096/20096 [==============================] - 55s 3ms/step - loss: 1.6196 - acc: 0.3647 - val_loss: 1.5269 - val_acc: 0.4101
Epoch 6/20
20096/20096 [==============================] - 52s 3ms/step - loss: 1.5863 - acc: 0.3828 - val_loss: 1.4915 - val_acc: 0.4298
Epoch 7/20
20096/20096 [==============================] - 56s 3ms/step - loss: 1.5605 - acc: 0.3958 - val_loss: 1.4615 - val_acc: